# VacationPy

In [1]:
# To dispay map 
#!jupyter nbextension enable --py gmaps


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress
from api_keys import weather_api_key
from api_keys import google_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load city data exported from WeatherPy Part I
weather_data_to_load = "../output_data/cities_part_I.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,315,alofi,58,NU,1602430333,100,-19.06,-169.92,22.00,6.20
1,497,eureka,1,US,1602430632,100,40.80,-124.16,10.00,1.50
2,444,uyaan,100,PH,1602430780,100,7.86,124.04,17.98,1.35
3,213,ulladulla,75,AU,1602430730,100,-35.35,150.47,14.44,2.60
4,224,aykhal,90,RU,1602430733,100,66.00,111.50,-9.00,1.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# location coordinates
coordinates = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)


# Access maps with unique API key
gmaps.configure(api_key=google_api_key)

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='2px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal

selection_df = pd.DataFrame(weather_data,\
                            columns = ["Country", "City", "Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness", "Humidity"])

# This data set temperature is in Celcius
ideal_choices_df = selection_df[(selection_df["Max Temp"] <= 27) & (selection_df["Max Temp"] > 20)\
                                & (selection_df["Wind Speed"] < 10)\
                                & (selection_df["Cloudiness"] == 0)]

ideal_choices_df


,Country,City,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity
204,PF,rikitea,-23.12,-134.97,22.90,6.41,0,80
222,AU,nhulunbuy,-12.23,136.77,24.00,2.10,0,78
223,OM,salalah,17.02,54.09,27.00,2.60,0,78
232,SO,hobyo,5.35,48.53,26.96,4.80,0,78
268,SO,bandarbeyla,9.49,50.81,26.48,2.69,0,74
356,BR,torres,-29.34,-49.73,20.59,6.74,0,63
364,IN,qadian,31.82,75.39,25.00,1.00,0,61
396,AU,karratha,-20.74,116.85,23.99,5.86,0,56
397,PT,ourem,39.64,-8.59,25.00,7.70,0,56
408,BR,ilhabela,-23.78,-45.36,23.07,3.16,0,54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_choices_df.copy()

hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
for i in hotel_df.index:
    lat = hotel_df["Lat"][i]
    lng = hotel_df["Lng"][i]
    
    params = {
            "location": f"{lat},{lng}",
            "radius": 5000,
            "types" : "hotel",
            "key": google_api_key
    }

    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #print(f"Hotel Name is {results[0]['name']}.")

    try:
        #print(f"Hotel Name is {results[0]['name']}.")
        hotel_df.loc[i, "Hotel Name"] = results[0]['name']
   
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")
        

hotel_df = hotel_df.dropna(how='any')
hotel_df.head()



,Country,City,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity,Hotel Name
204,PF,rikitea,-23.12,-134.97,22.90,6.41,0,80,Rikitea
222,AU,nhulunbuy,-12.23,136.77,24.00,2.10,0,78,Nhulunbuy
223,OM,salalah,17.02,54.09,27.00,2.60,0,78,Salalah
232,SO,hobyo,5.35,48.53,26.96,4.80,0,78,Hobyo
268,SO,bandarbeyla,9.49,50.81,26.48,2.69,0,74,Bandarbeyla


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
weights = hotel_df["Humidity"].astype(float)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type='TERRAIN')
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(locations,weights) 
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 20
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))